## Q1. Dynamic Arrays

In the dynamic arrays example in the lecture, we did not implement the `set` and `delete` interfaces. Implement them.

In [13]:
%%file dynamic_array2.c

#include <stdlib.h>
#include <string.h>
#include "dynamic_array.h"

int CAPACITY_INIT = 1;
int GROWTH_FACTOR = 2;

int DArray_init(DArray* arr){
    arr->array = (int *) malloc(CAPACITY_INIT*sizeof(int));
    if (arr->array == NULL) {
        return -1;
    }
    arr->upto = 0;
    arr->size = CAPACITY_INIT;
    return 1;
}

int append(DArray* arr, int i){
    int *iptr;
    if (arr->upto >= arr->size) {
        /* tmp pointer needed below as if allocation failed, original array would be lost */
        iptr = (int *) realloc(arr->array, arr->size*GROWTH_FACTOR*sizeof(int));
        if (iptr == NULL) {
            return -1;
        }
        arr->array = iptr;
        arr->size *= GROWTH_FACTOR;
    }
    arr->array[arr->upto] = i;
    return arr->upto++;
    
        
}

int get(DArray* arr, int index) {
    if (index >= arr->upto || index < 0) {
        return -1;
    }
    return arr->array[index];
}

int get_index(DArray* arr, int value) {
    int i;
    for (i=0; i< arr->upto; i++){
        if (arr->array[i]==value) {
            return i;
        }
    }
    return -1;
}

void DArray_free(DArray *arr) {
  free(arr->array);
}


/*your code here*/

int set(DArray* arr, int index, int value) {
    arr->array[index] = value;
    return 1;
}

int delete (DArray* arr, int value){
    int index = get_index(arr, value);
    int i;
    if (index == -1) {
        return -1;
    }
    for (i = index; i< arr->upto; i++){
        arr->array[i] = arr->array[i+1];
    }
    return arr->upto--;
    
}


Overwriting dynamic_array2.c


We can test our code with the following new "driver" file.

In [14]:
%%file dademo2.c

#include <stdio.h>
#include "dynamic_array.h"

int main() {
    DArray dynarray;
    DArray_init(&dynarray);
    printf("%d %d\n", dynarray.upto, dynarray.size);
    int i;
    for (i = 0; i < 200; i++) {
        append(&dynarray, i);
        printf("upto %d size %d val %d\n", dynarray.upto, dynarray.size, get(&dynarray, i));
    }
    for (i = 185; i < 190; i++) {
        set(&dynarray, i, i+1);
        printf("i %d val %d\n", i, get(&dynarray, i));
    }
    for (i = 180; i < 182; i++) {
        delete(&dynarray, i);
        printf("i %d val %d upto %d\n", i, get(&dynarray, i), dynarray.upto);
    }
    for (i = 180; i < 195; i++) {
        printf("val %d index %d\n", i, get_index(&dynarray, i));
    }
    DArray_free(&dynarray);
}

Overwriting dademo2.c


Compile and run it thus:

In [15]:
!gcc -o dademo2 dademo2.c dynamic_array2.c

In [16]:
!./dademo2

0 1
upto 1 size 1 val 0
upto 2 size 2 val 1
upto 3 size 4 val 2
upto 4 size 4 val 3
upto 5 size 8 val 4
upto 6 size 8 val 5
upto 7 size 8 val 6
upto 8 size 8 val 7
upto 9 size 16 val 8
upto 10 size 16 val 9
upto 11 size 16 val 10
upto 12 size 16 val 11
upto 13 size 16 val 12
upto 14 size 16 val 13
upto 15 size 16 val 14
upto 16 size 16 val 15
upto 17 size 32 val 16
upto 18 size 32 val 17
upto 19 size 32 val 18
upto 20 size 32 val 19
upto 21 size 32 val 20
upto 22 size 32 val 21
upto 23 size 32 val 22
upto 24 size 32 val 23
upto 25 size 32 val 24
upto 26 size 32 val 25
upto 27 size 32 val 26
upto 28 size 32 val 27
upto 29 size 32 val 28
upto 30 size 32 val 29
upto 31 size 32 val 30
upto 32 size 32 val 31
upto 33 size 64 val 32
upto 34 size 64 val 33
upto 35 size 64 val 34
upto 36 size 64 val 35
upto 37 size 64 val 36
upto 38 size 64 val 37
upto 39 size 64 val 38
upto 40 size 64 val 39
upto 41 size 64 val 40
upto 42 size 64 val 41
upto 43 size 64 val 42
upto 44 size 64 val 43
upto 45 siz

Copy the above output into the first answer on the google form. (http://goo.gl/forms/rH5he0YC44)

## 2. Linked List

If we put all our code into one file, we dont need to write the header file, since we use our interfaces in the same file. This does not makefor code usable by others, but as we hae seen ealier, is useful for quick playing and testing.

Provided below are parts of a partially implemented linked list in C. Implement the functions:

- `int get_index(Item* listptr, int value)` which returns the index at which the value is found, and otherwise -1.
- `Item* remove_item(Item* listptr, int value)`. Be careful to return NULL if the item with `value` was not in the list, and make sure that deletion repoints the pointer of the previous item to the next item.

In [18]:
%%file linked_list_incomplete.c

#include <stdlib.h>
#include <stdio.h>

typedef struct item {
    int value;
    struct item* rest;
} Item;

Item* new_item(int value){
    Item* newitem = (Item *) malloc(sizeof(Item));
    newitem->value = value;
    newitem->rest = NULL;
    return newitem;
}

Item* insert_front(Item* listptr, int value){
    Item* newitem = new_item(value);
    newitem->rest = listptr;
    return newitem;
}


int get(Item* listptr, int index){
    int ctr = 0;
    Item* p;
    for(p = listptr; p!= NULL; p = p->rest){
        if (ctr==index){
            return p->value;
        }
        ctr++;
    }
    return -1;
}


void free_all(Item* listptr) {
    Item *p;
    Item *next;
    for(p = listptr; p!= NULL; p = next){
        next = p->rest;
        free(p);
    }
}

int main(){
    Item* listptr;
    int i;
    listptr = new_item(0);
    for (i=1; i < 6; i++){
        listptr=insert_front(listptr, i);
    }
    for (i=0; i < 6; i++){
        printf("i %d Item %d\n", i, get(listptr, i));
    }
    listptr = remove_item(listptr, 3);
    for (i=0; i <= 5; i++){
        printf("i %d Item %d\n", i, get(listptr, i));
    }
    printf("Index for 3 %d\n", get_index(listptr, 3));
    free_all(listptr);
}

Add the two functions and write your code here.

In [179]:
%%file linked_list.c

//your code here

#include <stdlib.h>
#include <stdio.h>

typedef struct item {
    int value;
    struct item* rest;
} Item;

Item* new_item(int value){
    Item* newitem = (Item *) malloc(sizeof(Item));
    newitem->value = value;
    newitem->rest = NULL;
    return newitem;
}

Item* set_item(Item* listptr, Item* rest){
    /* This is the skeleton of an implementation; it doesn't do anything meaningful yet.*/
    listptr->rest = rest;
    return listptr;
}

Item* insert_front(Item* listptr, int value){
    Item* newitem = new_item(value);
    newitem->rest = listptr;
    return newitem;
}

Item* insert_back(Item* listptr, int value){
    Item* p = listptr;
    while(p->rest !=NULL){
        p = p->rest;
    }
    p->rest = new_item(value);
    return listptr;
}

int get(Item* listptr, int index){
    int ctr = 0;
    Item* p;
    for(p = listptr; p!= NULL; p = p->rest){
        if (ctr==index){
            return p->value;
        }
        ctr++;
    }
    return -1;
}

int get_index(Item* listptr, int value){
    /*returns the index at which the value is found, and otherwise -1.*/
    Item* p;
    int ctr = 0;
    Item* next;
    for(p = listptr; p!= NULL; p = next){
        if (p->value == value){
            return ctr;
        }
        next = p->rest;
        ctr++;
    }
    return -1;
}

Item* remove_item(Item* listptr, int value){
    /*Be careful to return NULL if the item with value was not in the list, and make sure that deletion 
    repoints the pointer of the previous item to the next item.*/
    Item* p;
    Item* newItem = new_item(0);
    int found = 0;
     
    for(p = listptr; p!= NULL; p = p->rest){
        while (p->value == value){
            p = p->rest;
            found = 1;
        }
        if (newItem->value == 0){
            newItem = new_item(p->value);
        } 
        else{
            newItem = insert_back(newItem,p->value);
        }
        
    }
    
    if (found==1){
        return newItem;
    } 
    else {
        return NULL;
    }
}
   
void free_all(Item* listptr) {
    Item *p;
    Item *next;
    for(p = listptr; p!= NULL; p = next){
        next = p->rest;
        free(p);
    }
}

int main(){
    Item* listptr;
    int i;
    listptr = new_item(0);
    for (i=1; i < 6; i++){
        listptr=insert_front(listptr, i);
    }
    for (i=0; i < 6; i++){
        printf("i %d Item%d\n", i, get(listptr, i));
    }
    listptr = remove_item(listptr, 3);
    for (i=0; i <= 5; i++){
        printf("i %d Item %d\n", i, get(listptr, i));
    }
    printf("Index for 3 %d\n", get_index(listptr, 3));
    free_all(listptr);
}

Overwriting linked_list.c


In [180]:
!gcc -o linked_list -g linked_list.c

In [181]:
!./linked_list

i 0 Item5
i 1 Item4
i 2 Item3
i 3 Item2
i 4 Item1
i 5 Item0
i 0 Item 5
i 1 Item 4
i 2 Item 2
i 3 Item 1
i 4 Item 0
i 5 Item -1
Index for 3 -1


Copy this output to the second answer on the google form. (http://goo.gl/forms/rH5he0YC44)